# **1. Corpus**

## Crawl URLs to extract all internal links 

**XENU Link Sleuth**  
https://home.snafu.de/tilman/xenulink.html 

*Le logiciel XENU Link Sleuth a finalement été retenu pour cette tâche* 

## Scrape textual data from crawled URLs
**BeautifulSoup HTML Parser**  
Réf : https://realpython.com/python-web-scraping-practical-introduction/

In [34]:
path = '../03-corpus/1-crawler/'
acteur = "pinel"
#pdfs = False

La liste des URLs à scrapper pour chaque corpus est contenue dans un fichier CSV. 
On commence donc par lire le CSV pour extraire nos URLS.

In [35]:
from pandas import *

regex = '.*png.*|.*jpeg.*|.*jpg.*|.*docx.*|.*js.*|.*font.*|.*gif.*|.*formulaire.*|.*?f%5B0%5D.*|.*img.*|.*%5Bfilter%.*|.*css.*|.*scripts.*|.*zip.*|.*xlsx.*|.*cms.*|.*/images/.*|.*sondage.*|.*/depenses/.*|.*demandes-acces.*'
# if not pdfs:
#     regex += '|.*pdf.*'

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur + '.csv', encoding='UTF-8') as f:
    csv = read_csv(f, sep=';')
    # Nettoyer ce qui ne devrait pas se trouver là
    csv = csv[~csv["Address"].str.contains(regex, case=False)] 
csv

,Address,Type,Title,Charset,Description
0,https://pinel.qc.ca/,text/html,Accueil - Institut national de psychiatrie lég...,utf-8,NaN
2,https://pinel.qc.ca/evenements/,text/html,Événements - Institut national de psychiatrie ...,utf-8,NaN
3,https://pinel.qc.ca/nouvelles/,text/html,Nouvelles - Institut national de psychiatrie l...,utf-8,NaN
4,https://pinel.qc.ca/covid-19/,text/html,COVID-19 - Institut national de psychiatrie lé...,utf-8,NaN
5,https://pinel.qc.ca/carriere/,text/html,Carrières - Institut national de psychiatrie l...,utf-8,NaN
...,...,...,...,...,...
686,https://pinel.qc.ca/wp-content/uploads/2022/04...,application/pdf,"<span class=""text"">Procès-verbal</span><span c...",utf-8,NaN
687,https://pinel.qc.ca/?p=3192,NaN,NaN,utf-8,NaN
688,https://pinel.qc.ca/?p=13292,NaN,NaN,utf-8,NaN
689,https://pinel.qc.ca/?p=18296,NaN,NaN,utf-8,NaN


In [36]:
fr = csv[~csv["Address"].str.contains('/en/')][['Address', 'Type']] # Données en français
en = csv[csv["Address"].str.contains('/en/')][['Address', 'Type']] # Données en anglais
print("On va tenter d'aspirer {} pages Web".format(len(fr) + len(en)))

fr

On va tenter d'aspirer 673 pages Web


,Address,Type
0,https://pinel.qc.ca/,text/html
2,https://pinel.qc.ca/evenements/,text/html
3,https://pinel.qc.ca/nouvelles/,text/html
4,https://pinel.qc.ca/covid-19/,text/html
5,https://pinel.qc.ca/carriere/,text/html
...,...,...
686,https://pinel.qc.ca/wp-content/uploads/2022/04...,application/pdf
687,https://pinel.qc.ca/?p=3192,NaN
688,https://pinel.qc.ca/?p=13292,NaN
689,https://pinel.qc.ca/?p=18296,NaN


In [37]:
en

,Address,Type


In [38]:
import requests, re, ssl, os, sys, pandas as pd
from bs4 import BeautifulSoup
#from requests.packages.urllib3.util.retry import Retry

def getTextHTML(url):    
    html = requests.get(url, headers = {'User-Agent': 'My User Agent 1.0'}, verify=False)
    html.encoding = 'utf-8'
    html = html.text

    soup = BeautifulSoup(html, "html.parser")
    tags_to_remove = ['head', 'header', 'script', 'footer', 'nav', 'form'] # Enlever 'form' pour le site du CHU Ste Justine

    # Classes CSS spécifiques aux différents sites
    attr_to_remove = ['div[class="contenu-fluide piv"]', 'div[role="navigation"]', 'div[class="section__wrapper section__wrapper--padding tac grid--inline-block"]', 'div[id="slidebox"]',
    'div[class="col-md-12 mise-a-jour"]', 'h1[class="sr-only"]', 'a[class="cd-top js-cd-top"]', 'div[class="nocontent"]', 'p[class="footer-lien-resonances"]', 'p[class="suivre"]',
    'section[class="field field-name-field-date-de-mise-jour field-type-datetime field-label-inline clearfix view-mode-full"]', 'a[class="active"]', 'p[class="footer-resonances"]',
    'div[class="item-list item-list-pager"]', 'ul[class="pub-solr-sub-menu"]', 'a[href="#main-content"]', 'div[id="block-sociauxcrchum"]', 'div[class="visually-hidden"]',
    'div[id="Breadcrumb"]', 'div[id="pageInfo"]', 'div[id="breadcrumb"]', 'div[class="pagesCreation"]', 'a[href="#contenu"]', 'div[class="bandeau"]',
    'div[id="seeAlso"]', 'a[href="/nous-ecrire.aspx"]', 'li[class="CMSListMenuLI"]', 'li[class="CMSListMenuLI navFirst"]', 'li[class="CMSListMenuLI navLast"]',
    'div[class="alert alert-danger"]', 'span[class="alertoverflow"]', 'div[class="alert alert-warning alert-dismissible"]', 'ul[class="menu"]', 'div[id="letters-filter"]',
    'ul[class="pager"]', 'a[href="#main-menu"]', 'ul[class="custom_menu"]', 'h2[class="element-invisible"]', 'div[class="breadcrumb"]',
    'a[class="all-cta"]', 'div[class="sub-menu-inner container"]', 'div[class="fixed-dk-nav"]', 'div[class="fixed-dk-nav-container"]', 'div[class="container-inner"]',
    'div[class="socials"]', 'div[class="breadcrumbs"]', 'a[class="btn-print"]', 'ul[class="list-buttons"]', 'p[class="visually-hidden"]', 
    'a[class="back-to-top"]', 'a[class="sr-only sr-only-focusable"]', 'ol[class="breadcrumb"]', 'div[class="container-fluid piv_bas"]', 'div[class="col-12 formBasPage"]',
    'div[class="container-fluid rangee-footer"]', 'a[class="visuallyHidden passerContenu"]', 'div[id="bandeau-alerte"]', 'div[class="menu-sec-wrapper col-12 col-lg-12"]',
    'a[href="#layout-content"]', 'div[class="paragraph feedback"]', 'p[class="last-update"]', 'ul[class="footer__menu--list"]', 'div[class="footer__info"]',
    'section[class="hello-bar"]', 'section[class="breadcrumb"]', 'div[class="menu-page"]', 'div[class="no-print menu-non-voyant"]', 'div[class="navigation"]',
    'div[class="pure-bloc pure-u-1 pure-u-md-1-3 pure-u-lg-1-4 side-menu"]', 'div[class="pied"]', 'div[class="social"]', 'div[class="piv-bas"]',
    'div[class="partage"]', 'div[class="pied-print no-screen"]', 'div[class="carte dynamic-carte-interactive-display ui-carte-panel"]',
    'div[id="piv"]', 'ul[class="social-nav top-bar-social"]', 'div[class="sidebar"]', 'ul[class="side-menu"]', 'div[class="zoom-button-wrapper"]',
    'a[href="#maincontent"]', 'a[href="#content"]', 'p[id="breadcrumbs"]', 'div[class="mega-menu-wrap"]', 'div[class="menu_2"]', 'div[class="welcome"]',
    'div[class="header_two"]', 'div[class="footer"]', 'div[class="footer-wrapper"]', 'div[class="custom-accessibility-tools js-only"]', 'section[role="navigation"]',
    'div[class="container-fluid container-blue container-dl-menu"]', 'div[class="col-xs-12 dl-menuwrapper menu-mobile visible-x"]']
    
    for t in tags_to_remove:
        tags = soup.find_all(t)
        for tag in tags:
            tag.decompose()

    for t in attr_to_remove:
        attr = soup.select(t)
        for a in attr:
            a.decompose()


    data = soup.get_text(separator=' ').replace("\n", " ").replace("\r", " ") 
    data = re.compile(r"\s+").sub(" ", data).strip()
    
    return data

In [39]:
import requests, re, ssl, os, sys, pandas as pd
from bs4 import BeautifulSoup
#from requests.packages.urllib3.util.retry import Retry

def getTextHTML(url):    
    html = requests.get(url, headers = {'User-Agent': 'My User Agent 1.0'}, verify=False)
    html.encoding = 'utf-8'
    html = html.text

    soup = BeautifulSoup(html, "html.parser")
    tags_to_remove = ['head', 'header', 'script', 'footer', 'nav', 'form'] # Enlever 'form' pour le site du CHU Ste Justine

    # Classes CSS spécifiques aux différents sites
    attr_to_remove = ['div[class="contenu-fluide piv"]', 'div[role="navigation"]', 'div[class="section__wrapper section__wrapper--padding tac grid--inline-block"]', 'div[id="slidebox"]',
    'div[class="col-md-12 mise-a-jour"]', 'h1[class="sr-only"]', 'a[class="cd-top js-cd-top"]', 'div[class="nocontent"]', 'p[class="footer-lien-resonances"]', 'p[class="suivre"]',
    'section[class="field field-name-field-date-de-mise-jour field-type-datetime field-label-inline clearfix view-mode-full"]', 'a[class="active"]', 'p[class="footer-resonances"]',
    'div[class="item-list item-list-pager"]', 'ul[class="pub-solr-sub-menu"]', 'a[href="#main-content"]', 'div[id="block-sociauxcrchum"]', 'div[class="visually-hidden"]',
    'div[id="Breadcrumb"]', 'div[id="pageInfo"]', 'div[id="breadcrumb"]', 'div[class="pagesCreation"]', 'a[href="#contenu"]', 'div[class="bandeau"]',
    'div[id="seeAlso"]', 'a[href="/nous-ecrire.aspx"]', 'li[class="CMSListMenuLI"]', 'li[class="CMSListMenuLI navFirst"]', 'li[class="CMSListMenuLI navLast"]',
    'div[class="alert alert-danger"]', 'span[class="alertoverflow"]', 'div[class="alert alert-warning alert-dismissible"]', 'ul[class="menu"]', 'div[id="letters-filter"]',
    'ul[class="pager"]', 'a[href="#main-menu"]', 'ul[class="custom_menu"]', 'h2[class="element-invisible"]', 'div[class="breadcrumb"]',
    'a[class="all-cta"]', 'div[class="sub-menu-inner container"]', 'div[class="fixed-dk-nav"]', 'div[class="fixed-dk-nav-container"]', 'div[class="container-inner"]',
    'div[class="socials"]', 'div[class="breadcrumbs"]', 'a[class="btn-print"]', 'ul[class="list-buttons"]', 'p[class="visually-hidden"]', 
    'a[class="back-to-top"]', 'a[class="sr-only sr-only-focusable"]', 'ol[class="breadcrumb"]', 'div[class="container-fluid piv_bas"]', 'div[class="col-12 formBasPage"]',
    'div[class="container-fluid rangee-footer"]', 'a[class="visuallyHidden passerContenu"]', 'div[id="bandeau-alerte"]', 'div[class="menu-sec-wrapper col-12 col-lg-12"]',
    'a[href="#layout-content"]', 'div[class="paragraph feedback"]', 'p[class="last-update"]', 'ul[class="footer__menu--list"]', 'div[class="footer__info"]',
    'section[class="hello-bar"]', 'section[class="breadcrumb"]', 'div[class="menu-page"]', 'div[class="no-print menu-non-voyant"]', 'div[class="navigation"]',
    'div[class="pure-bloc pure-u-1 pure-u-md-1-3 pure-u-lg-1-4 side-menu"]', 'div[class="pied"]', 'div[class="social"]', 'div[class="piv-bas"]',
    'div[class="partage"]', 'div[class="pied-print no-screen"]', 'div[class="carte dynamic-carte-interactive-display ui-carte-panel"]',
    'div[id="piv"]', 'ul[class="social-nav top-bar-social"]', 'div[class="sidebar"]', 'ul[class="side-menu"]', 'div[class="zoom-button-wrapper"]',
    'a[href="#maincontent"]', 'a[href="#content"]', 'p[id="breadcrumbs"]', 'div[class="mega-menu-wrap"]', 'div[class="menu_2"]', 'div[class="welcome"]',
    'div[class="header_two"]', 'div[class="footer"]', 'div[class="footer-wrapper"]', 'div[class="custom-accessibility-tools js-only"]', 'section[role="navigation"]',
    'div[class="container-fluid container-blue container-dl-menu"]', 'div[class="col-xs-12 dl-menuwrapper menu-mobile visible-x"]']
    
    for t in tags_to_remove:
        tags = soup.find_all(t)
        for tag in tags:
            tag.decompose()

    for t in attr_to_remove:
        attr = soup.select(t)
        for a in attr:
            a.decompose()

    data = soup.get_text(separator=' ').replace("\n", " ").replace("\r", " ") 
    data = re.compile(r"\s+").sub(" ", data).strip()
    
    return data

In [40]:
import io
from PyPDF2 import PdfReader

def getTextPDF(url):
    pdf_link = requests.get(url)
    with io.BytesIO(pdf_link.content) as f:
        reader = PdfReader(f)
        nb_pages = len(reader.pages)
        text = ''
        if nb_pages <= 15: # On ne prendra pas les documents de plus de 15 pages
            for i in range(nb_pages):
                page = reader.pages[i]
                text += page.extract_text().lower().replace('\n', '')
        if text :
            return text


In [41]:
def scrape_list(df):
    output = []
    pdfs = df[df['Type'] == 'application/pdf']['Address'].tolist()
    htmls = df[df['Type'] == 'text/html']['Address'].tolist() 
    # Il y a aussi des pages spour lesquelles le crawler n'a pas été en mesure de consigner une valeur
    # On va essayer d'aller chercher leur contenu avec la fonction getTextHTML 
    nans = df[df["Type"].isnull()]['Address'].tolist()  
    htmls = htmls + nans

    for url in htmls :
        try: 
            text = getTextHTML(url)
            if not '���' in text:
                output.append({'Address': url, 'type': 'text/html', 'text':text})
        except Exception as e:
            print("ERROR " + " - " + url)
            print(e)

    for url in pdfs :
        try: 
            text = getTextPDF(url)
            if not '���' in text:
                output.append({'Address': url, 'type': 'application/pdf', 'text':text})
        except Exception as e:
            print("ERROR " + " - " + url)
            print(e)

    return output

In [42]:
sites_fr = scrape_list(fr)
if(len(en) > 0):
    sites_en = scrape_list(en)

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pinel.qc.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pinel.qc.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pinel.qc.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#s

ERROR  - https://pinel.qc.ca/wp-content/uploads/2020/07/GestionCOVID_reponse-rapide-VF.pdf
argument of type 'NoneType' is not iterable
ERROR  - https://pinel.qc.ca/wp-content/uploads/2020/11/Gestion-COVID-English.pdf
argument of type 'NoneType' is not iterable
ERROR  - https://pinel.qc.ca/wp-content/uploads/2020/08/Pre%CC%81sentationMIDI_COVID_20200714.pdf
argument of type 'NoneType' is not iterable
ERROR  - https://pinel.qc.ca/wp-content/uploads/2021/10/Rapport-annuel-2020-2021_CLPQS_INPL-Philippe-Pinel.pdf
argument of type 'NoneType' is not iterable
ERROR  - https://pinel.qc.ca/wp-content/uploads/2022/03/Code-dethique-et-deontologie_20220223.pdf
argument of type 'NoneType' is not iterable
ERROR  - https://pinel.qc.ca/wp-content/uploads/2020/03/CA_reglement_regie_interne-20200219-class%C3%A9-dans-Publication_r%C3%A8glement.pdf
argument of type 'NoneType' is not iterable
ERROR  - https://pinel.qc.ca/wp-content/uploads/2021/03/Rapport_ACT_SIM_forensiques-juillet-2020.pdf
argument of typ

 impossible to decode XFormObject /Meta5
 impossible to decode XFormObject /Meta14


ERROR  - https://pinel.qc.ca/wp-content/uploads/2020/11/PV_CA20200909.pdf
argument of type 'NoneType' is not iterable
ERROR  - https://pinel.qc.ca/wp-content/uploads/2020/12/PV_CA20201028_Huis-clos-caviarde.pdf
argument of type 'NoneType' is not iterable
ERROR  - https://pinel.qc.ca/wp-content/uploads/2022/07/PV_CA20220406_caviarde.pdf
argument of type 'NoneType' is not iterable
ERROR  - https://pinel.qc.ca/wp-content/uploads/2022/04/PV_CA20220223.pdf
argument of type 'NoneType' is not iterable


In [43]:
output_path = '../03-corpus/2-data/'

sites_fr = pd.DataFrame(sites_fr)
sites_fr = csv.merge(sites_fr, how='right', on='Address')
sites_fr = sites_fr[['Address', 'Title', 'Type', 'text']]

sites_fr

,Address,Title,Type,text
0,https://pinel.qc.ca/,Accueil - Institut national de psychiatrie lég...,text/html,L’Institut national de psychiatrie légale Phil...
1,https://pinel.qc.ca/evenements/,Événements - Institut national de psychiatrie ...,text/html,Événements L'Institut national de psychiatrie ...
2,https://pinel.qc.ca/nouvelles/,Nouvelles - Institut national de psychiatrie l...,text/html,Nouvelles Les journalistes qui désirent réalis...
3,https://pinel.qc.ca/covid-19/,COVID-19 - Institut national de psychiatrie lé...,text/html,COVID-19 Quelques outils et ressources pour vo...
4,https://pinel.qc.ca/carriere/,Carrières - Institut national de psychiatrie l...,text/html,Carrières Vous pouvez faire une différence. Co...
...,...,...,...,...
612,https://pinel.qc.ca/wp-content/uploads/2022/02...,Dépliant &#8211; Clinique d&rsquo;évaluation d...,application/pdf,clinique d’évaluation durisque de violenceresp...
613,https://pinel.qc.ca/wp-content/uploads/2022/06...,"<span class=""text"">Ordre du jour</span><span c...",application/pdf,séance régulière du conseil d’administration ...
614,https://pinel.qc.ca/wp-content/uploads/2022/07...,"<span class=""text"">Procès-verbal</span><span c...",application/pdf,procès -verbal de la séance spéciale du ca de...
615,https://pinel.qc.ca/wp-content/uploads/2022/04...,"<span class=""text"">Ordre du jour</span><span c...",application/pdf,séance régulière du conseil d’administration...


In [44]:
sites_fr.to_csv(output_path + '1-fr/' + acteur + '.csv', escapechar='/')
if(len(en) > 0):
    sites_en = pd.DataFrame(sites_en)
    sites_en = csv.merge(sites_en, how='right', on='Address')
    sites_en = sites_en[['Address', 'Title', 'Type', 'text']]
    sites_en.to_csv(output_path + '1-en/' + acteur + '_en.csv', escapechar='/')